In [ ]:
!pip install alpha-vantage

In [ ]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta

In [ ]:
def fetch_intraday_data(symbol, interval, start_date, end_date, api_key):
    """
    Fetches intraday data month-by-month and returns it as a DataFrame.
    """
    all_data = []
    current_date = start_date

    while current_date <= end_date:
        # Format month as YYYY-MM to use it in the API
        month_str = current_date.strftime('%Y-%m')
        print(f"Fetching data for {month_str}")

        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&apikey={api_key}&month={month_str}&datatype=csv&outputsize=full'
        response = requests.get(url)

        # Load data into DataFrame and filter by month
        if response.status_code == 200 and 'timestamp' in response.text:
            monthly_data = pd.read_csv(StringIO(response.text))

            # Ensure 'timestamp' column is present
            if 'timestamp' in monthly_data.columns:
                monthly_data['timestamp'] = pd.to_datetime(monthly_data['timestamp'])
                # Filter data for the correct date range
                monthly_data = monthly_data[(monthly_data['timestamp'] >= current_date) &
                                            (monthly_data['timestamp'] < current_date + timedelta(days=31))]
                all_data.append(monthly_data)
            else:
                print(f"No 'timestamp' column found for {month_str}")
        else:
            print(f"Failed to fetch data for {month_str}. Status code: {response.status_code}")

        # Move to the next month
        current_date += timedelta(days=31)
        current_date = current_date.replace(day=1)

    # Combine all months' data into a single DataFrame
    return pd.concat(all_data, ignore_index=True)

## For fetching train and test data

In [ ]:
api_key = 'EWIPHBQVF2CH0WGQ'
symbol = 'AAPL'
interval = '1min'

# Define date ranges for train and test sets
train_start_date = datetime(2020, 1, 1)
train_end_date = datetime(2022, 1, 31)

train_data = fetch_intraday_data(symbol, interval, train_start_date, train_end_date, api_key)

# Now fetching test data:
api_key = 'WZP0EF92X2IDW3HL'
test_start_date = datetime(2022, 2, 1)
test_end_date = datetime(2022, 6, 30)

test_data = fetch_intraday_data(symbol, interval, test_start_date, test_end_date, api_key)
test_data.to_csv('AAPL_test_data.csv', index=False)

Fetching data for 2020-01
Fetching data for 2020-02
Fetching data for 2020-03
Fetching data for 2020-04
Fetching data for 2020-05
Fetching data for 2020-06
Fetching data for 2020-07
Fetching data for 2020-08
Fetching data for 2020-09
Fetching data for 2020-10
Fetching data for 2020-11
Fetching data for 2020-12
Fetching data for 2021-01
Fetching data for 2021-02
Fetching data for 2021-03
Fetching data for 2021-04
Fetching data for 2021-05
Fetching data for 2021-06
Fetching data for 2021-07
Fetching data for 2021-08
Fetching data for 2021-09
Fetching data for 2021-10
Fetching data for 2021-11
Fetching data for 2021-12
Fetching data for 2022-01
Fetching data for 2022-02
Fetching data for 2022-03
Fetching data for 2022-04
Fetching data for 2022-05
Fetching data for 2022-06


In [ ]:
train_data.to_csv('AAPL_train_data.csv', index=False)

print("Data saved to AAPL_train_data.csv and AAPL_test_data.csv")

Data saved to AAPL_train_data.csv and AAPL_test_data.csv


In [ ]:
print(train_data.head())

            timestamp     open     high      low    close  volume
0 2020-01-31 19:59:00  74.6878  74.6878  74.6515  74.6878   24020
1 2020-01-31 19:58:00  74.6636  74.6805  74.6515  74.6805    4640
2 2020-01-31 19:57:00  74.6612  74.6636  74.6515  74.6515   20300
3 2020-01-31 19:56:00  74.6539  74.6539  74.6515  74.6515    2864
4 2020-01-31 19:55:00  74.6515  74.6636  74.6515  74.6636    7800


In [ ]:
print(test_data.head())

            timestamp      open      high       low     close  volume
0 2022-02-28 19:59:00  162.3063  162.3556  162.3063  162.3556     473
1 2022-02-28 19:58:00  162.3556  162.3556  162.3556  162.3556     200
2 2022-02-28 19:57:00  162.3556  162.3556  162.3556  162.3556     100
3 2022-02-28 19:56:00  162.3950  162.4048  162.3950  162.4048     359
4 2022-02-28 19:55:00  162.3162  162.3260  162.3162  162.3260     579


In [ ]:
def aggregate_by_date(data):
    # Extract date from timestamp
    data['date'] = data['timestamp'].dt.date

    # Group by date and aggregate price data
    aggregated_data = data.groupby('date').agg({
        'open': list,
        'high': list,
        'low': list,
        'close': list,
        'volume': list
    }).reset_index()

    return aggregated_data

# Aggregate training data by date
train_aggregated = aggregate_by_date(train_data)
train_aggregated.to_csv('AAPL_train_data_aggregated.csv', index=False)
print("Aggregated training data saved to AAPL_train_data_aggregated.csv")

# Aggregate test data by date
test_aggregated = aggregate_by_date(test_data)
test_aggregated.to_csv('AAPL_test_data_aggregated.csv', index=False)
print("Aggregated test data saved to AAPL_test_data_aggregated.csv")

Aggregated training data saved to AAPL_train_data_aggregated.csv
Aggregated test data saved to AAPL_test_data_aggregated.csv


In [ ]:
train_aggregated.head()

,date,open,high,low,close,volume
0,2020-01-02,"[72.9863, 72.9548, 72.9524, 72.9427, 72.9427, ...","[73.0009, 72.9791, 72.9548, 72.9548, 72.9548, ...","[72.9863, 72.9548, 72.9524, 72.9427, 72.9427, ...","[73.0009, 72.9791, 72.9548, 72.9548, 72.9548, ...","[5200, 14492, 908, 1880, 1360, 4080, 400, 4012..."
1,2020-01-03,"[72.0702, 72.0774, 72.0629, 72.0823, 72.0823, ...","[72.0702, 72.0774, 72.0799, 72.0823, 72.0823, ...","[72.0629, 72.0774, 72.0629, 72.0605, 72.0823, ...","[72.0629, 72.0774, 72.0799, 72.0605, 72.0823, ...","[3208, 400, 11208, 6024, 3844, 18092, 1200, 36..."
2,2020-01-06,"[72.6155, 72.5913, 72.6204, 72.6228, 72.6349, ...","[72.6276, 72.5937, 72.6204, 72.6228, 72.6349, ...","[72.6155, 72.5913, 72.6058, 72.6228, 72.6228, ...","[72.6204, 72.5913, 72.6058, 72.6228, 72.6228, ...","[15528, 6808, 1600, 3200, 2180, 400, 5232, 120..."
3,2020-01-07,"[70.9674, 70.9189, 70.9189, 70.9237, 70.9165, ...","[70.9989, 70.9552, 70.9431, 70.931, 70.9431, 7...","[70.9068, 70.8947, 70.9189, 70.9068, 70.8922, ...","[70.9189, 70.9552, 70.9189, 70.9189, 70.931, 7...","[24820, 24832, 14104, 17364, 20512, 18712, 121..."
4,2020-01-08,"[73.6771, 73.6698, 73.6723, 73.6747, 73.6771, ...","[73.682, 73.682, 73.6723, 73.6747, 73.6771, 73...","[73.6747, 73.6698, 73.6723, 73.6747, 73.6601, ...","[73.6795, 73.682, 73.6723, 73.6747, 73.6747, 7...","[10700, 15304, 720, 1400, 9260, 19124, 2000, 7..."


In [ ]:
test_aggregated.head()

,date,open,high,low,close,volume
0,2022-02-01,"[171.6532, 171.7418, 171.7024, 171.6926, 171.7...","[171.7811, 171.7418, 171.7418, 171.7221, 171.7...","[171.6434, 171.6434, 171.6434, 171.6434, 171.7...","[171.6434, 171.6434, 171.7418, 171.6532, 171.7...","[2997, 1901, 2093, 1514, 1419, 5696, 3751, 573..."
1,2022-02-02,"[170.994, 170.9546, 170.9546, 171.0826, 171.08...","[171.0038, 170.994, 170.9546, 171.1022, 171.08...","[170.9546, 170.9546, 170.9546, 171.0038, 171.0...","[170.9546, 170.9939, 170.9546, 171.1022, 171.0...","[2303, 3789, 1155, 4476, 250, 473, 5113, 2713,..."
2,2022-02-03,"[171.3679, 171.299, 171.2695, 171.2006, 171.27...","[171.3679, 171.3679, 171.2793, 171.2793, 171.2...","[171.3089, 171.2989, 171.2694, 171.2006, 171.1...","[171.3679, 171.3679, 171.2793, 171.2793, 171.2...","[12187, 4253, 1301, 4153, 20727, 2596, 2605, 5..."
3,2022-02-04,"[169.9414, 169.9414, 169.9414, 169.9808, 169.9...","[169.9414, 169.9807, 169.9414, 169.9808, 169.9...","[169.9414, 169.9414, 169.9414, 169.9808, 169.9...","[169.9414, 169.9807, 169.9414, 169.9808, 169.9...","[496, 2276, 150, 100, 875, 272, 100, 805, 500,..."
4,2022-02-07,"[169.8428, 169.8429, 169.9118, 169.8921, 169.8...","[169.8428, 169.8429, 169.9118, 169.8921, 169.8...","[169.8428, 169.8428, 169.9118, 169.8921, 169.8...","[169.8428, 169.8428, 169.9118, 169.8921, 169.8...","[200, 220, 100, 120, 1429, 446, 100, 2300, 226..."


In [ ]:
print(len(train_aggregated))
print(len(test_aggregated))

525
104


In [ ]:
import pandas as pd
import ast  # For safely parsing list-like strings

def aggregate_by_date(data):
    # Ensure the 'close' column contains lists, not strings
    data['close'] = data['close'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    # Extract the last closing price from the 'close' column
    data['last_close'] = data['close'].apply(lambda x: x[-1] if isinstance(x, list) else None)

    # Shift the last_close column to compare with the next day
    data['label'] = (data['last_close'].shift(-1) > data['last_close']).astype(int)

    # Drop the last row since it won't have a label
    data = data[:-1]

    return data

# Read the CSV files
train_aggregated = pd.read_csv('AAPL_train_data_aggregated.csv')
test_aggregated = pd.read_csv('AAPL_test_data_aggregated.csv')

# Apply the label creation function
train_aggregated = aggregate_by_date(train_aggregated)
test_aggregated = aggregate_by_date(test_aggregated)

# Save the updated CSV files
train_aggregated.to_csv('AAPL_train_data_aggregated_label.csv', index=False)
test_aggregated.to_csv('AAPL_test_data_aggregated_label.csv', index=False)

print("Updated CSV files saved.")


Updated CSV files saved.


In [ ]:
train_aggregated.head(20)

,date,open,high,low,close,volume,last_close,label
0,2020-01-02,"[72.9863, 72.9548, 72.9524, 72.9427, 72.9427, ...","[73.0009, 72.9791, 72.9548, 72.9548, 72.9548, ...","[72.9863, 72.9548, 72.9524, 72.9427, 72.9427, ...","[73.0009, 72.9791, 72.9548, 72.9548, 72.9548, ...","[5200, 14492, 908, 1880, 1360, 4080, 400, 4012...",71.5200,1
1,2020-01-03,"[72.0702, 72.0774, 72.0629, 72.0823, 72.0823, ...","[72.0702, 72.0774, 72.0799, 72.0823, 72.0823, ...","[72.0629, 72.0774, 72.0629, 72.0605, 72.0823, ...","[72.0629, 72.0774, 72.0799, 72.0605, 72.0823, ...","[3208, 400, 11208, 6024, 3844, 18092, 1200, 36...",71.7939,0
2,2020-01-06,"[72.6155, 72.5913, 72.6204, 72.6228, 72.6349, ...","[72.6276, 72.5937, 72.6204, 72.6228, 72.6349, ...","[72.6155, 72.5913, 72.6058, 72.6228, 72.6228, ...","[72.6204, 72.5913, 72.6058, 72.6228, 72.6228, ...","[15528, 6808, 1600, 3200, 2180, 400, 5232, 120...",71.3067,1
3,2020-01-07,"[70.9674, 70.9189, 70.9189, 70.9237, 70.9165, ...","[70.9989, 70.9552, 70.9431, 70.931, 70.9431, 7...","[70.9068, 70.8947, 70.9189, 70.9068, 70.8922, ...","[70.9189, 70.9552, 70.9189, 70.9189, 70.931, 7...","[24820, 24832, 14104, 17364, 20512, 18712, 121...",72.8094,0
4,2020-01-08,"[73.6771, 73.6698, 73.6723, 73.6747, 73.6771, ...","[73.682, 73.682, 73.6723, 73.6747, 73.6771, 73...","[73.6747, 73.6698, 73.6723, 73.6747, 73.6601, ...","[73.6795, 73.682, 73.6723, 73.6747, 73.6747, 7...","[10700, 15304, 720, 1400, 9260, 19124, 2000, 7...",71.8666,1
5,2020-01-09,"[75.2332, 75.2332, 75.221, 75.2162, 75.2162, 7...","[75.2332, 75.2332, 75.221, 75.2162, 75.2162, 7...","[75.2283, 75.2332, 75.221, 75.2162, 75.2162, 7...","[75.2332, 75.2332, 75.221, 75.2162, 75.2162, 7...","[27076, 2000, 2000, 1124, 5572, 6244, 400, 400...",74.0528,1
6,2020-01-10,"[75.3059, 75.3059, 75.3059, 75.3156, 75.3131, ...","[75.3059, 75.3059, 75.3059, 75.3156, 75.318, 7...","[75.2962, 75.2938, 75.3059, 75.3156, 75.3131, ...","[75.2986, 75.2938, 75.3059, 75.3156, 75.318, 7...","[9704, 4052, 4780, 400, 5760, 2652, 720, 3920,...",75.4586,1
7,2020-01-13,"[77.2449, 77.2546, 77.2449, 77.2109, 77.2449, ...","[77.2643, 77.2618, 77.2546, 77.2521, 77.2449, ...","[77.2328, 77.2376, 77.2449, 77.2109, 77.2449, ...","[77.2643, 77.2376, 77.2449, 77.2449, 77.2449, ...","[28952, 16328, 6664, 30844, 7980, 10720, 2780,...",75.5070,1
8,2020-01-14,"[75.5967, 75.5943, 75.5943, 75.5434, 75.5119, ...","[75.5967, 75.5967, 75.5943, 75.5604, 75.5386, ...","[75.5604, 75.5579, 75.5725, 75.5434, 75.5119, ...","[75.5604, 75.5725, 75.5725, 75.5604, 75.5386, ...","[17176, 3988, 1164, 5740, 9520, 1116, 800, 204...",76.6050,0
9,2020-01-15,"[75.6719, 75.6573, 75.6452, 75.6452, 75.6597, ...","[75.6719, 75.6694, 75.6452, 75.6452, 75.6597, ...","[75.6452, 75.6452, 75.6452, 75.6379, 75.6549, ...","[75.6694, 75.6694, 75.6452, 75.6379, 75.6573, ...","[7180, 12216, 400, 4280, 1408, 400, 1096, 1100...",75.5943,1


In [ ]:
test_aggregated.head(20)

,date,open,high,low,close,volume,last_close,label
0,2022-02-01,"[171.6532, 171.7418, 171.7024, 171.6926, 171.7...","[171.7811, 171.7418, 171.7418, 171.7221, 171.7...","[171.6434, 171.6434, 171.6434, 171.6434, 171.7...","[171.6434, 171.6434, 171.7418, 171.6532, 171.7...","[2997, 1901, 2093, 1514, 1419, 5696, 3751, 573...",172.2239,0
1,2022-02-02,"[170.994, 170.9546, 170.9546, 171.0826, 171.08...","[171.0038, 170.994, 170.9546, 171.1022, 171.08...","[170.9546, 170.9546, 170.9546, 171.0038, 171.0...","[170.9546, 170.9939, 170.9546, 171.1022, 171.0...","[2303, 3789, 1155, 4476, 250, 473, 5113, 2713,...",171.3974,0
2,2022-02-03,"[171.3679, 171.299, 171.2695, 171.2006, 171.27...","[171.3679, 171.3679, 171.2793, 171.2793, 171.2...","[171.3089, 171.2989, 171.2694, 171.2006, 171.1...","[171.3679, 171.3679, 171.2793, 171.2793, 171.2...","[12187, 4253, 1301, 4153, 20727, 2596, 2605, 5...",171.3974,1
3,2022-02-04,"[169.9414, 169.9414, 169.9414, 169.9808, 169.9...","[169.9414, 169.9807, 169.9414, 169.9808, 169.9...","[169.9414, 169.9414, 169.9414, 169.9808, 169.9...","[169.9414, 169.9807, 169.9414, 169.9808, 169.9...","[496, 2276, 150, 100, 875, 272, 100, 805, 500,...",171.4290,0
4,2022-02-07,"[169.8428, 169.8429, 169.9118, 169.8921, 169.8...","[169.8428, 169.8429, 169.9118, 169.8921, 169.8...","[169.8428, 169.8428, 169.9118, 169.8921, 169.8...","[169.8428, 169.8428, 169.9118, 169.8921, 169.8...","[200, 220, 100, 120, 1429, 446, 100, 2300, 226...",169.8330,0
5,2022-02-08,"[172.4831, 172.4535, 172.4732, 172.4732, 172.4...","[172.5718, 172.483, 172.4732, 172.5028, 172.44...","[172.483, 172.4043, 172.4535, 172.4732, 172.41...","[172.5718, 172.483, 172.4535, 172.5028, 172.44...","[1637, 2178, 436, 353, 200, 600, 200, 330, 149...",169.6064,1
6,2022-02-09,"[173.3894, 173.3894, 173.35, 173.3402, 173.389...","[173.3894, 173.3894, 173.35, 173.3402, 173.399...","[173.3894, 173.3894, 173.35, 173.3106, 173.340...","[173.3894, 173.3894, 173.35, 173.3106, 173.340...","[100, 1295, 100, 635, 1093, 4841, 400, 1000, 1...",173.2811,0
7,2022-02-10,"[169.2419, 169.1926, 169.1926, 169.2025, 169.1...","[169.2419, 169.1926, 169.2222, 169.2025, 169.1...","[169.1631, 169.1926, 169.1532, 169.0547, 169.1...","[169.2419, 169.1926, 169.2222, 169.0547, 169.1...","[1118, 389, 1071, 2369, 554, 3457, 300, 575, 6...",173.1432,0
8,2022-02-11,"[165.6756, 165.6756, 165.6854, 165.6559, 165.7...","[165.7051, 165.6756, 165.6854, 165.6559, 165.7...","[165.646, 165.6559, 165.646, 165.6165, 165.705...","[165.7051, 165.6756, 165.6559, 165.6165, 165.7...","[6033, 1270, 3163, 3005, 1591, 1500, 3504, 362...",168.5424,0
9,2022-02-14,"[166.3652, 166.3159, 166.2962, 166.2963, 166.2...","[166.3652, 166.3159, 166.2962, 166.2963, 166.3...","[166.3652, 166.3159, 166.2962, 166.2963, 166.2...","[166.3652, 166.3159, 166.2962, 166.2963, 166.3...","[1444, 1850, 250, 100, 835, 196, 7261, 466, 24...",163.7939,1


In [ ]:
import pandas as pd
import json
import os
from datetime import datetime
from glob import glob
import ast
import re

In [ ]:
# Clone repo if not exists
if not os.path.exists('sn2'):
    !git clone https://github.com/koa-fin/sn2.git

Cloning into 'sn2'...
remote: Enumerating objects: 81318, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 81318 (delta 0), reused 2 (delta 0), pack-reused 81315 (from 1)
Receiving objects: 100% (81318/81318), 147.74 MiB | 14.21 MiB/s, done.
Resolving deltas: 100% (12217/12217), done.
Updating files: 100% (81242/81242), done.


In [ ]:
def clean_tweet_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www.\S+', '', text)

    # Remove special characters but keep $ and numbers
    text = re.sub(r'[^A-Za-z0-9\s$.]', ' ', text)

    # Remove AT_USER
    text = text.replace('AT_USER', '')

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text.lower()

def process_tweet_files():
    base_path = 'sn2/tweet/preprocessed/AAPL/'
    daily_tweets = {}

    for file_name in os.listdir(base_path):
        try:
            date = datetime.strptime(file_name, '%Y-%m-%d')
            file_path = os.path.join(base_path, file_name)
            tweets_for_day = []

            with open(file_path, 'r', encoding='utf-8') as file:
                for line in file:
                    try:
                        tweet = json.loads(line)
                        tweet_text = ' '.join(tweet['text'])
                        cleaned_text = clean_tweet_text(tweet_text)
                        if cleaned_text.strip():  # Only add if not empty after cleaning
                            tweets_for_day.append(cleaned_text)
                    except (json.JSONDecodeError, KeyError):
                        continue

            if tweets_for_day:
                daily_tweets[date.strftime('%Y-%m-%d')] = ', '.join(tweets_for_day)

        except ValueError:
            continue

    df = pd.DataFrame(list(daily_tweets.items()), columns=['date', 'text'])
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')

    train_start = '2020-01-01'
    train_end = '2022-01-31'
    test_start = '2022-02-01'
    test_end = '2022-06-30'

    train_data = df[(df['date'] >= train_start) & (df['date'] <= train_end)]
    test_data = df[(df['date'] >= test_start) & (df['date'] <= test_end)]

    train_data.to_csv('AAPL_tweets_train.csv', index=False)
    test_data.to_csv('AAPL_tweets_test.csv', index=False)

    return train_data, test_data

In [ ]:
# Process the data
train_tweets, test_tweets = process_tweet_files()

print("\nTrain data sample:")
print(train_tweets.head())
print("\nTest data sample:")
print(test_tweets.head())


Train data sample:
           date                                               text
1017 2020-01-01  rt at user a look at earnings for january 2020...
917  2020-01-02  $ aapl 300, $ aapl new article apple revives r...
341  2020-01-03  $ aapl new article wait for a better price on ...
425  2020-01-04  rt at user apple is up over 73 000 since its i...
76   2020-01-05  $ aapl new article apple stock is really expen...

Test data sample:
           date                                               text
887  2022-02-01  rt at user $ amc fails to deliver skyrocket 1 ...
1083 2022-02-02  rt at user 02 02 watchlist $ aapl over 175.75 ...
919  2022-02-03  rt at user . at user called this spread trade ...
547  2022-02-04  rt at user $ googl now worth 10 of us gdp . $ ...
365  2022-02-05  rt at user i like how no one is talking about ...


In [ ]:
print(len(train_tweets))
print(len(test_tweets))

762
150
